# Welcome to your first assignment!

Instructions are below. Please give this a try, and look in the solutions folder if you get stuck (or feel free to ask me!)

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">Just before we get to the assignment --</h2>
            <span style="color:#f71;">I thought I'd take a second to point you at this page of useful resources for the course. This includes links to all the slides.<br/>
            <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">https://edwarddonner.com/2024/11/13/llm-engineering-resources/</a><br/>
            Please keep this bookmarked, and I'll continue to add more useful links there over time.
            </span>
        </td>
    </tr>
</table>

# HOMEWORK EXERCISE ASSIGNMENT

Upgrade the day 1 project to summarize a webpage to use an Open Source model running locally via Ollama rather than OpenAI

You'll be able to use this technique for all subsequent projects if you'd prefer not to use paid APIs.

**Benefits:**
1. No API charges - open-source
2. Data doesn't leave your box

**Disadvantages:**
1. Significantly less power than Frontier Model

## Recap on installation of Ollama

Simply visit [ollama.com](https://ollama.com) and install!

Once complete, the ollama server should already be running locally.  
If you visit:  
[http://localhost:11434/](http://localhost:11434/)

You should see the message `Ollama is running`.  

If not, bring up a new Terminal (Mac) or Powershell (Windows) and enter `ollama serve`  
And in another Terminal (Mac) or Powershell (Windows), enter `ollama pull llama3.2`  
Then try [http://localhost:11434/](http://localhost:11434/) again.

If Ollama is slow on your machine, try using `llama3.2:1b` as an alternative. Run `ollama pull llama3.2:1b` from a Terminal or Powershell, and change the code below from `MODEL = "llama3.2"` to `MODEL = "llama3.2:1b"`

In [1]:
# imports

import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display

In [2]:
# Constants

OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}
MODEL = "llama3.2"

In [3]:
# Create a messages list using the same format that we used for OpenAI

messages = [
    {"role": "user", "content": "Describe some of the business applications of Generative AI"}
]

In [4]:
payload = {
        "model": MODEL,
        "messages": messages,
        "stream": False
    }

In [7]:
response = requests.post(OLLAMA_API, json=payload, headers=HEADERS)
print(Markdown(response.json()['message']['content']))

<IPython.core.display.Markdown object>


# Introducing the ollama package

And now we'll do the same thing, but using the elegant ollama python package instead of a direct HTTP call.

Under the hood, it's making the same call as above to the ollama server running at localhost:11434

In [9]:
import ollama

response = ollama.chat(model=MODEL, messages=messages)
print(Markdown(response['message']['content']))

<IPython.core.display.Markdown object>


# NOW the exercise for you

Take the code from day1 and incorporate it here, to build a website summarizer that uses Llama 3.2 running locally instead of OpenAI

In [11]:
# imports

import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
import ollama

# If you get an error running this cell, then please head over to the troubleshooting notebook!

In [12]:
# A class to represent a Webpage
# If you're not familiar with Classes, check out the "Intermediate Python" notebook

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [13]:
# Let's try one out. Change the website and add print statements to follow along.

ed = Website("https://edwarddonner.com")
print(ed.title)
print(ed.text)

Home - Edward Donner
Home
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of AI startup untapt,
acquired in 2021
.
We work with groundbreaking, proprietary LLMs verticalized for talent, we’ve
patented
our matching model, and our award-winning platform has happy customers and tons of press coverage.
Connect
with me for

In [14]:
# Define our system prompt - you can experiment with this later, changing the last sentence to 'Respond in markdown in Spanish."

system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [15]:
# A function that writes a User Prompt that asks for summaries of websites:

def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [16]:
print(user_prompt_for(ed))

You are looking at a website titled Home - Edward Donner
The contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.

Home
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of AI startup untapt,
acquired in 2021
.

In [17]:
# See how this function creates exactly the format above

def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [18]:
# Try this out, and then try for a few more websites

messages_for(ed)

[{'role': 'system',
  'content': 'You are an assistant that analyzes the contents of a website and provides a short summary, ignoring text that might be navigation related. Respond in markdown.'},
 {'role': 'user',
  'content': 'You are looking at a website titled Home - Edward Donner\nThe contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.\n\nHome\nOutsmart\nAn arena that pits LLMs against each other in a battle of diplomacy and deviousness\nAbout\nPosts\nWell, hi there.\nI’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (\nvery\namateur) and losing myself in\nHacker News\n, nodding my head sagely to things I only half understand.\nI’m the co-founder and CTO of\nNebula.io\n. We’re applying AI to a field where it can make a massive, posi

In [25]:
# And now: call the OpenAI API. You will get very familiar with this!

def summarize(url):
    website = Website(url)
    response = ollama.chat(
        model = "llama3.2",
        messages = messages_for(website)
    )
    return Markdown(response['message']['content'])

In [26]:
summarize("https://edwarddonner.com")

**Summary**

* The website belongs to Edward Donner, a co-founder and CTO of Nebula.io, an AI startup that applies AI to help people discover their potential.
* The company uses proprietary LLMs (Large Language Models) for talent matching and has patented its matching model.
* The website features various resources for mastering AI and LLM engineering, including articles, toolkits, and resources.

**News and Announcements**

* **Mastering AI and LLM Engineering – Resources** (November 13, 2024): A collection of resources to help software engineers transition into AI data scientists.
* **From Software Engineer to AI Data Scientist – resources** (October 16, 2024): A set of resources for those looking to enter the field of AI data science.
* **Outsmart LLM Arena – a battle of diplomacy and deviousness** (June 26, 2024): An arena where LLMs compete in a game of diplomacy and strategy.
* **Choosing the Right LLM: Toolkit and Resources** (August 6, 2024): A toolkit and resources for selecting the right LLM for various use cases.

In [27]:
summarize("https://cisasif.blogspot.com")

### Embedded Systems Website Summary

#### Overview

This website appears to be a blog focused on embedded systems, covering various topics related to programming, design, and development. It includes sections for specific projects (e.g., AVR-I, L8: Timer/Counter), as well as general categories such as Low Power Design and BLDC.

#### Recent Posts

* **DevOps #1: Docker for Embedded Engineers** (November 13, 2024)
	+ Introduction to Docker for embedded engineers
	+ Benefits of using Docker in embedded systems
* **Low Power Design #2: Visualizing Power Consumption** (September 02, 2024)
	+ Techniques for visualizing power consumption in embedded systems
	+ Dynamic solution for analyzing efficiency
* **Low Power Design #1: Optimizing Battery Life** (July 22, 2024)
	+ Hardware best practices for optimizing battery life in embedded systems

#### Archives

The website includes archives by month, year, and category. The archive section shows a list of articles published from various dates.

Note: This summary only includes information that is not related to navigation or meta data.

In [28]:
summarize("https://antechnologiesofficial.com")

### A.N.Technologies Website Summary

#### Overview
A.N.Technologies appears to be an online education platform offering various courses and programs in different subjects, including coding, mathematics, physics, chemistry, English, and more.

#### Courses and Programs
The website lists numerous courses with varying durations and validity periods. Some of the notable programs include:

* **Core Simon 101 Courses** (5 years validity)
* **LoveToCodeOnline** (3-2 year validity)
* **Great Physics** (3 years validity)
* **H Level Chemistry** (3 years validity)

#### Validity Periods
Courses and programs have different validity periods ranging from 1 to 6 years.

#### Exclusive Products
The website mentions several exclusive products, including:

* **Heiser Coding Club**
* **Orbital Boost (Channel)**

Note that the website's structure is mostly navigation-related, with "More Courses", "SignUp", "Login" being prominent sections. The content seems to be focused on showcasing various courses and programs available on the platform.

In [ ]:
summarize("https://www.st.com")